## Introdução

A deriva genética é um importante fenômeno evolutivo responsável por causar variações nas frequências alélicas de um determinado gene de uma população ao longo do tempo. Devido ao seu caráter estocástico, o efeito da deriva genética é mais forte quanto menor o tamanho da população sobre a qual ela atua. Para melhor entender como funciona esse mecanismo e como ele será simulado a seguir, vejamos o seguinte exemplo:

Dois alelos, *A* e *a*, são equifrequentes numa população de 10 organismos diploides. A cada geração, parte dessa população se reproduz, gerando descendência, e parte morre. O percentual dessa população que morre a cada geração (S) é idêntico ao percentual que é renovado por novos nascimentos, de modo que o tamanho da população (N = 10) é constante ao longo do tempo. Para esse exemplo, vamos também assumir que S = 1, o que significa que a população será totalmente renovada a cada geração.

O efeito da deriva genética nesse caso nada mais é do que a incerteza associada ao processo de amostragem. Cada vez que amostramos aleatóriamente gametas para gerar os descendentes da população, a frequência dos alelos *A* e *a* variam levemente ao redor de uma média (a média nesse caso é a frequência dos alelos na geração passada). Como a estimativa das frequências alélicas da geração seguinte dependem somente das frequências na geração atual, essa variação pode progressivamente se acumular, causando alterações significativas no panorama genético da população ao longo prazo.

## Simulação de deriva genética

Baseado no que determinamos previamente, podemos seguir uma série de passos que nos permitirão simular o fenômeno de deriva genética. São eles:

1. Inicializar uma população com tamanho N e alelos equifrequentes;
2. Para cada geração:
  1. Gerar S x N descendentes por cruzamentos aleatórios (amostragem com reposição);
  2. Eliminar (1 - S) x N organismos da geração anterior (amostragem sem reposição);
  3. Concatenar os descendentes e sobreviventes numa nova população;
  4. Avaliar estatisticamente as novas frequências alélicas;
3. Quando o número desejado de gerações se passou ou um dos alelos foi fixado, parar a simulação e apresentar os resultados.

Onde:

+ N = tamanho da população diploide;
+ S = percentual (0-1) da população renovada a cada geração.

No entanto, podemos ainda simplificar esse algoritmo ao presumir que a população se encontra em equilíbrio de [Hardy-Weinberg](https://pt.wikipedia.org/wiki/Equil%C3%ADbrio_de_Hardy-Weinberg). Essa simplicação é possível porque os nossos cruzamentos ocorrem ao acaso na população, de modo que não é necessário simular cada organismo diploide independentemente. Ao invés disso, podemos resumir a nossa população diploide aos seus gametas, tal que o tamanho da população gamética seja igual a 2 x N. O restante dos passos do algoritmo permanecem iguais.

## Resultados esperados

Esperamos que o efeito da deriva genética seja mais forte:

1. Quanto menor for o tamanho da população (N);
2. Quanto maior for a taxa de substituição da população (S).

Compare os resultados a seguir, simulados ambos com 3 alelos iniciais por 200 gerações, mas com populações de tamanhos diferentes:

![Imgur](https://i.imgur.com/mdrqJ1y.png)

Os resultados seguem o que esperávamos? Experimente alterar outros parâmetros no simulador para ver como isso impacta o efeito da deriva genética.

## Simulador

In [2]:
#@title Clique no botão "▶" abaixo para rodar o simulador e depois clique em "Iniciar" para começar uma simulação com os parâmetros selecionados
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, FloatSlider, FloatLogSlider, Button, IntProgress, Label, HBox, Output


def simulate(pop, allele_freqs, sub_rate, gens, n_runs):
    alleles = tuple(range(len(allele_freqs)))
    runs = []
    for i in range(n_runs):
        run_progress.value = i
        run_label.value = f"Iteração atual: {i}/{run_progress.max}"
        new_pop = np.array(alleles * (2 * pop // len(alleles)) + alleles[:2 * pop % len(alleles)])
        stats = [np.bincount(new_pop, minlength=len(alleles)) / (2 * pop)]
        while 1 not in stats[-1] and (gens == -1 or len(stats) < gens):
            if len(alleles) != len(stats[-1]):
                print(i, alleles, stats[-1])
            descendents = np.random.choice(alleles, round(2 * pop * sub_rate), p=stats[-1])
            survivors = np.random.choice(new_pop, round(2 * pop * (1 - sub_rate)), replace=False)
            new_pop = np.concatenate((descendents, survivors))
            stats.append(np.bincount(new_pop, minlength=len(alleles)) / (2 * pop))
        runs.append(stats)
    run_progress.value = run_progress.max
    run_label.value = f"Iteração atual: {run_progress.max}/{run_progress.max}"
    return np.array(max(runs, key=len))


def plot(btn):
    run_progress.max = run_slider.value
    progress_box.layout.visibility = "visible"
    stats = simulate(round(pop_slider.value),
                     [1/allele_slider.value] * allele_slider.value,
                     sub_slider.value,
                     gen_slider.value, 
                     run_slider.value)
    fig, axes = plt.subplots(1, 2)
    fig.set_size_inches(12.8, 2.4)
    x = range(len(stats))
    axes[0].plot(x, np.where(stats != 0, stats, np.nan))
    axes[0].set_ylim((0, 1))
    axes[0].autoscale(tight=True, axis="x")
    axes[1].stackplot(x, *np.transpose(stats))
    axes[1].autoscale(tight=True)
    fig.suptitle(f"População: {round(pop_slider.value)} | "
                 f"Taxa de subs.: {sub_slider.value} | "
                 f"Gerações: {len(stats)}",
                 fontsize="medium",
                 fontweight="bold")
    with out:
        plt.show()
    progress_box.layout.visibility = "hidden"


pop_slider = FloatLogSlider(min=0, max=6, value=1000)
allele_slider = IntSlider(min=1, max=10, value=3)
sub_slider = FloatSlider(min=0, max=1, value=1)
gen_slider = IntSlider(min=-1, max=100_000, value=200)
run_slider = IntSlider(min=1, max=50, value=1)
show_btn = Button(description="Iniciar")
clear_btn = Button(description="Limpar")
run_label = Label(value=f"Iteração atual: 0/{run_slider.value}")
run_progress = IntProgress()
out = Output()
progress_box = HBox([run_label, run_progress])
progress_box.layout.visibility = "hidden"
    
show_btn.on_click(plot)
clear_btn.on_click(lambda btn: out.clear_output())
display(Label("População:"),
        pop_slider,
        Label("Núm. de alelos:"),
        allele_slider,
        Label("Taxa de substituição da pop.:"),
        sub_slider,
        Label("Máx. de gerações (-1 para só interromper quando ocorrer fixação):"),
        gen_slider,
        Label("Melhor de quantas iterações (seleciona a mais longa dentre as iterações):"),
        run_slider,
        HBox([show_btn, clear_btn]),
        out,
        progress_box)

Label(value='População:')

FloatLogSlider(value=1000.0, max=6.0)

Label(value='Núm. de alelos:')

IntSlider(value=3, max=10, min=1)

Label(value='Taxa de substituição da pop.:')

...

Label(value='Máx. de gerações (-1 para só interromper quando ocorrer fixação):')

IntSlider(value=200, max=100000, min=-1)

Label(value='Melhor de quantas iterações (seleciona a mais longa dentre as iterações):')

IntSlider(value=1, max=50, min=1)

Output()